# Tool Permissions

Tool permissions define authority boundaries for what an agent can observe or mutate. This notebook demonstrates two approaches: filtering tools before agent construction, and enforcing permissions at runtime.

In [ ]:
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.tools import (
    ToolPermission,
    ToolPermissionError,
    tool_permission,
    get_permissions,
    filter_tools_by_permission,
    enforce_tools_permissions,
)

## Define Tools

Tools are annotated with required permissions using the `@tool_permission` decorator. Without the decorator, tools default to READ permission.

In [ ]:
@tool_permission(ToolPermission.READ)
def get_balance(account_id: str) -> float:
    """Get the current balance of an account."""
    print(f"Reading balance for account: {account_id}")
    return 1500.00


@tool_permission(ToolPermission.READ)
def get_transactions(account_id: str, limit: int = 10) -> list[dict]:
    """Get recent transactions for an account."""
    print(f"Reading transactions for account: {account_id}")
    return [{"id": "tx1", "amount": -50.00, "description": "Coffee shop"}]


@tool_permission(ToolPermission.WRITE)
def transfer_funds(from_account: str, to_account: str, amount: float) -> bool:
    """Transfer funds between accounts."""
    print(f"Transferring ${amount} from {from_account} to {to_account}")
    return True


@tool_permission(ToolPermission.WRITE, ToolPermission.CONNECT)
def send_payment_notification(email: str, amount: float) -> bool:
    """Send payment notification to external email."""
    print(f"Sending notification to {email} for ${amount}")
    return True


ALL_TOOLS = [get_balance, get_transactions, transfer_funds, send_payment_notification]

In [ ]:
for tool in ALL_TOOLS:
    print(f"{tool.__name__}: {get_permissions(tool)}")

## Construction-Time Filtering

Filter tools before passing them to the agent. The agent only sees tools it has permission to use.

### Read-only agent

In [ ]:
read_only_tools = filter_tools_by_permission(ALL_TOOLS, granted={ToolPermission.READ})
print(f"Tools available: {[t.__name__ for t in read_only_tools]}")

In [ ]:
agent = get_agent(tools=read_only_tools)
result, _ = await run_agent(agent, "What is the balance of account ACC-123?", verbose=True)
print(f"\nAnswer: {result.result.output}")

If asked to transfer funds, the agent cannot comply because it has no access to the transfer tool:

In [ ]:
result, _ = await run_agent(agent, "Transfer $100 from ACC-123 to ACC-456", verbose=True)
print(f"\nAnswer: {result.result.output}")

### Read-write agent

In [ ]:
write_tools = filter_tools_by_permission(ALL_TOOLS, granted={ToolPermission.READ, ToolPermission.WRITE})
print(f"Tools available: {[t.__name__ for t in write_tools]}")

In [ ]:
agent = get_agent(tools=write_tools)
result, _ = await run_agent(agent, "Transfer $100 from ACC-123 to ACC-456", verbose=True)
print(f"\nAnswer: {result.result.output}")

### Full access agent

In [ ]:
full_tools = filter_tools_by_permission(
    ALL_TOOLS, 
    granted={ToolPermission.READ, ToolPermission.WRITE, ToolPermission.CONNECT}
)
print(f"Tools available: {[t.__name__ for t in full_tools]}")

In [ ]:
agent = get_agent(tools=full_tools)
result, _ = await run_agent(agent, "Transfer $100 from ACC-123 to ACC-456 and notify user@example.com", verbose=True)
print(f"\nAnswer: {result.result.output}")

## Runtime Enforcement

An alternative approach: the agent sees all tools, but execution fails if permission is denied. Use `enforce_tools_permissions` to wrap tools with permission checks baked in.

### Agent with all tools visible, read-only enforcement

In [ ]:
enforced_tools = enforce_tools_permissions(ALL_TOOLS, granted={ToolPermission.READ})
print(f"Tools visible to agent: {[t.__name__ for t in enforced_tools]}")

In [ ]:
agent = get_agent(tools=enforced_tools)
result, _ = await run_agent(agent, "What is the balance of account ACC-123?", verbose=True)
print(f"\nAnswer: {result.result.output}")

The agent sees transfer_funds and may try to use it, but execution raises ToolPermissionError:

In [ ]:
result, _ = await run_agent(agent, "Transfer $100 from ACC-123 to ACC-456", verbose=True)
print(f"\nAnswer: {result.result.output}")

### Agent with write enforcement

In [ ]:
enforced_tools = enforce_tools_permissions(ALL_TOOLS, granted={ToolPermission.READ, ToolPermission.WRITE})
agent = get_agent(tools=enforced_tools)
result, _ = await run_agent(agent, "Transfer $100 from ACC-123 to ACC-456", verbose=True)
print(f"\nAnswer: {result.result.output}")

## Comparison

**Construction-time filtering** (`filter_tools_by_permission`): The agent never sees disallowed tools. Cleaner, but the agent cannot explain why a capability is unavailable.

**Runtime enforcement** (`enforce_tools_permissions`): The agent sees all tools but execution fails if denied. The agent can attempt the action and receive an error, potentially explaining the limitation to the user.